In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense, Activation, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Flipkart_Amazon Mobile Reviews - Flipkart_Amazon Mobile Reviews.csv')

In [ ]:
data.columns

Index(['Unnamed: 0', 'Review-Title', 'rating', 'Review-Body', 'Product Name'], dtype='object')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23777 entries, 0 to 23776
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    23777 non-null  int64 
 1   Review-Title  23775 non-null  object
 2   rating        23777 non-null  object
 3   Review-Body   23160 non-null  object
 4   Product Name  23777 non-null  object
dtypes: int64(1), object(4)
memory usage: 928.9+ KB


**Preprocessing**

In [ ]:
data.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
data['rating'] = data['rating'].apply(lambda x: float(list(x.split(' '))[0]))

In [ ]:
data

,Review-Title,rating,Review-Body,Product Name
0,Worst phone ever,1.0,Hang problem,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12..."
1,Ok !!! Not up to the mark,2.0,I'm writing this review after using 3days !!!B...,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12..."
2,Awesome look,5.0,Camera is so good n very fast phone back look ...,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12..."
3,One plus losing is originality!!!,3.0,The media could not be loaded.\n ...,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12..."
4,Read,1.0,I got my delivery on 23 feb when I unboxed the...,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12..."
...,...,...,...,...
23772,Batten in this rangee,4.0,Must he Bhai iss range me bas cemra thoda sa s...,"vivo iQOO Z6 5G (Chromatic Blue, 6GB RAM, 128G..."
23773,Good mobile,4.0,Good product,"vivo iQOO Z6 5G (Chromatic Blue, 6GB RAM, 128G..."
23774,Good camera,5.0,Good,"vivo iQOO Z6 5G (Chromatic Blue, 6GB RAM, 128G..."
23775,Warest mobile,1.0,Don't buy this mobile phone money weast screen...,"vivo iQOO Z6 5G (Chromatic Blue, 6GB RAM, 128G..."


In [ ]:
models = list(data['Product Name'].value_counts().index)

In [ ]:
data.dropna(inplace = True)

**Sentiment Analysis**

In [ ]:
!pip install textblob

In [ ]:
from textblob import TextBlob

In [ ]:
def polarity(text):
    return TextBlob(text).sentiment.polarity

In [ ]:
polarity('Worst Shirt')

-1.0

In [ ]:
polarity(data['Review-Title'][0])

-1.0

In [ ]:
data['Review-Title'].isna().sum()

2

In [ ]:
data['polarity_score'] = data['Review-Title'].apply(lambda x : polarity(x))

def sentiment(x):
    if x<0:
        return 'negative'
    elif x==0:
        return 'neutral'
    else:
        return 'positive'

data['polarity'] = data['polarity_score'].map(lambda x: sentiment(x))

In [ ]:
data

,Review-Title,rating,Review-Body,Product Name,polarity_score,polarity
0,Worst phone ever,1.0,Hang problem,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",-1.000000,negative
1,Ok !!! Not up to the mark,2.0,I'm writing this review after using 3days !!!B...,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",0.976562,positive
2,Awesome look,5.0,Camera is so good n very fast phone back look ...,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",1.000000,positive
3,One plus losing is originality!!!,3.0,The media could not be loaded.\n ...,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",0.000000,neutral
4,Read,1.0,I got my delivery on 23 feb when I unboxed the...,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",0.000000,neutral
...,...,...,...,...,...,...
23772,Batten in this rangee,4.0,Must he Bhai iss range me bas cemra thoda sa s...,"vivo iQOO Z6 5G (Chromatic Blue, 6GB RAM, 128G...",0.000000,neutral
23773,Good mobile,4.0,Good product,"vivo iQOO Z6 5G (Chromatic Blue, 6GB RAM, 128G...",0.700000,positive
23774,Good camera,5.0,Good,"vivo iQOO Z6 5G (Chromatic Blue, 6GB RAM, 128G...",0.700000,positive
23775,Warest mobile,1.0,Don't buy this mobile phone money weast screen...,"vivo iQOO Z6 5G (Chromatic Blue, 6GB RAM, 128G...",0.000000,neutral


**Sentiment-Rating Correlation**

In [ ]:
data[data['polarity'] == 'negative'].rating.mean()

1.8968609865470851

In [ ]:
data[data['polarity'] == 'positive'].rating.mean()

4.351416868678426

In [ ]:
data[data['polarity'] == 'neutral'].rating.mean()

3.005220328573622

In [ ]:
data[data['Product Name'] == models[0]]['rating'].mean()

3.398876404494382

In [ ]:
models

['Redmi 9 Activ (Carbon Black, 4GB RAM, 64GB Storage)',
 'OPPO A31 (Mystery Black, 6GB RAM, 128GB Storage)',
 'Redmi 10 Prime (Bifrost Blue 4GB RAM 64GB ROM',
 'OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 128GB Storage)',
 'Redmi Note 11 (Space Black, 4GB RAM, 64GB Storage)',
 'realme narzo 50A (Oxygen Blue , 4GB RAM + 64 GB Storage)',
 'vivo iQOO Z6 5G (Chromatic Blue, 6GB RAM, 128GB Storage)',
 'Samsung Galaxy M32']

In [ ]:
model_dict = {}

In [ ]:
model_dict[models[0]] = data[data['Product Name'] == models[0]]['rating'].mean()

In [ ]:
model_dict[models[1]] = data[data['Product Name'] == models[1]]['rating'].mean()
model_dict[models[2]] = data[data['Product Name'] == models[2]]['rating'].mean()
model_dict[models[3]] = data[data['Product Name'] == models[3]]['rating'].mean()
model_dict[models[4]] = data[data['Product Name'] == models[4]]['rating'].mean()
model_dict[models[5]] = data[data['Product Name'] == models[5]]['rating'].mean()
model_dict[models[6]] = data[data['Product Name'] == models[6]]['rating'].mean()
model_dict[models[7]] = data[data['Product Name'] == models[7]]['rating'].mean()

In [ ]:
model_dict

{'Redmi 9 Activ (Carbon Black, 4GB RAM, 64GB Storage)': 3.398876404494382,
 'OPPO A31 (Mystery Black, 6GB RAM, 128GB Storage)': 3.7231453311490754,
 'Redmi 10 Prime (Bifrost Blue 4GB RAM 64GB ROM': 3.251042535446205,
 'OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 128GB Storage)': 3.727620730270907,
 'Redmi Note 11 (Space Black, 4GB RAM, 64GB Storage)': 3.291445874337623,
 'realme narzo 50A (Oxygen Blue , 4GB RAM + 64 GB Storage)': 3.6557142857142857,
 'vivo iQOO Z6 5G (Chromatic Blue, 6GB RAM, 128GB Storage)': 3.2989010989010987,
 'Samsung Galaxy M32': 3.180389221556886}